# Data Preprocessing

In [1]:
import numpy as np
import librosa
def get_melspectrogram_db(file_path, sr=16000, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
    wav,sr = librosa.load(file_path,sr=sr)
    if wav.shape[0]<5*sr:
        wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:5*sr]
    spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,
              hop_length=hop_length,n_mels=n_mels,fmin=fmin,fmax=fmax)
    spec_db=librosa.power_to_db(spec,top_db=top_db)
    return spec_db

In [2]:
def spec_to_image(spec, eps=1e-6):
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)
    return spec_scaled

# Additional Imports

In [3]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# Create Dataset from Cough Files

In [4]:
# Import data from cough dataset
import os
from torch.utils.data import Dataset

POSITIVE_FOLDER = "pos"
NEGATIVE_FOLDER = "neg"

ROOT = "cough_data"

class CoughDataSet(Dataset):
    def __init__(self, root=ROOT, dataset_type="train"):
        self.root = root
        positive_coughs = []
        negative_coughs = []
        for entry in os.scandir(os.path.join(self.root, os.path.join(dataset_type, POSITIVE_FOLDER))):
            if entry.is_file():
                positive_coughs.append(entry.path)
        for entry in os.scandir(os.path.join(self.root, os.path.join(dataset_type, NEGATIVE_FOLDER))):
            if entry.is_file():
                negative_coughs.append(entry.path)
        data = [] # will contain tuples
        labels = []
        for filename in positive_coughs:
            # Load image
            mel_db = get_melspectrogram_db(filename)
            # Transform image using spec_to_image and get_melspectrogram_db
            spec_img = spec_to_image(mel_db)
            data.append(spec_img[np.newaxis,...])
            labels.append(0)
        for filename in negative_coughs:
            # Load image
            mel_db = get_melspectrogram_db(filename)
            # Transform image using spec_to_image and get_melspectrogram_db
            spec_img = spec_to_image(mel_db)
            data.append(spec_img[np.newaxis,...])
            labels.append(1)
        self.data = data
        self.labels = labels
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        return self.data[index], self.labels[index]
        

In [5]:
train_data = CoughDataSet()
test_data = CoughDataSet(dataset_type="test")
valid_data = CoughDataSet(dataset_type="val")
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True)

# Initialize Metrics

In [6]:
from ignite.metrics import Accuracy, Precision, Recall
from ignite.contrib.metrics import RocCurve

# Setup Training Metrics
training_metrics = [Accuracy()]

for metric in training_metrics:
    metric.reset()

# Setup Test Metrics
metrics = [Accuracy(), Precision(average=False), Recall(average=False), ROC_AUC()]

# Add F1 Score as a Metric
F1 = (metrics[1] * metrics[2] * 2 / (metrics[1] + metrics[2])).mean()
metrics.append(F1)

for metric in metrics:
    metric.reset()

## Setup Tensorboard Logging

In [7]:
from torch.utils.tensorboard import SummaryWriter

summary_writer = SummaryWriter("experiments", comment=ROOT)

# Model

In [8]:
from torchvision.models import resnet34
import torch
import torch.nn as nn
import torch.optim as optim
if torch.cuda.is_available():
    device=torch.device('cuda:0')
else:
    device=torch.device('cpu')
resnet_model = resnet34(pretrained=True)
resnet_model.fc = nn.Linear(512,2)
resnet_model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
resnet_model = resnet_model.to(device)

In [9]:
learning_rate = 2e-4
start_epoch = 1
epochs = 50
loss_fn = nn.CrossEntropyLoss()
resnet_train_losses=[]
resnet_valid_losses=[]

In [10]:
optimizer = optim.Adam(resnet_model.parameters(), lr=learning_rate)

In [ ]:
## IN ORDER TO USE AN EXISTING CHECKPOINT FOR CONTINUED TRAINING, RUN THIS BLOCK
checkpoint_epoch = 50
checkpoint = torch.load('checkpoints/{}_{}.pth'.format(ROOT, checkpoint_epoch))
resnet_model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']
# loss = checkpoint['loss']

In [11]:
def lr_decay(optimizer, epoch):
    if epoch%10==0:
        new_lr = learning_rate / (10**(epoch//10))
        optimizer = setlr(optimizer, new_lr)
        print(f'Changed learning rate to {new_lr}')
    return optimizer

In [12]:
def train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, train_losses, valid_losses, change_lr=None):
    for epoch in tqdm(range(start_epoch,epochs+1)):
        model.train()
        batch_losses=[]
        if change_lr:
            optimizer = change_lr(optimizer, epoch)
            
        # First train the model through training dataset
        train_accuracy = training_metrics[0]
        for i, data in enumerate(train_loader):
            print("Index: {}".format(i))
            x, y = data
            optimizer.zero_grad()
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            # Compute accuracy
            train_accuracy.update((y_hat, y))
            # Compute loss
            loss = loss_fn(y_hat, y)
            loss.backward()
            batch_losses.append(loss.item())
            optimizer.step()
        train_losses.append(batch_losses)
        
        # Add Training Accuracy to Summary
        computed_train_accuracy = train_accuracy.compute()
        summary_writer.add_scalar("Training Accuracy", computed_train_accuracy, epoch)
        
        # Print Info
        print(f'Epoch - {epoch} Train-Loss : {np.mean(train_losses[-1])} Train-Accuracy: {computed_train_accuracy}')
        
        # Reset Training Accuracy for later use
        train_accuracy.reset()
        
        model.eval()
        batch_losses=[]
        trace_y = []
        trace_yhat = []

        # Then evaluate on validation data
        for i, data in enumerate(valid_loader):
            x, y = data
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            trace_y.append(y.cpu().detach().numpy())
            trace_yhat.append(y_hat.cpu().detach().numpy())      
            batch_losses.append(loss.item())
            # Calculate metrics here on validation data
            for metric in metrics:
                metric.update((torch.from_numpy(trace_yhat[-1]), torch.from_numpy(trace_y[-1])))
        valid_losses.append(batch_losses)
        trace_y = np.concatenate(trace_y)
        trace_yhat = np.concatenate(trace_yhat)
        
        # Compute metrics here on validation data
        print("Metric Results:")
        for metric in metrics:
            metric_name = metric.__class__.__name__
            computed_metric = metric.compute()
            if (torch.is_tensor(computed_metric) and len(computed_metric.size()) > 0):
                # Must iterate because this is a tensor
                for idx, row in enumerate(computed_metric):
                    summary_writer.add_scalar(metric_name + f' of {idx}', row, epoch)
            else:
                # Add Metric to Summary
                summary_writer.add_scalar(metric_name, computed_metric, epoch)
            print("Validation {} is {}".format(metric_name, computed_metric))
        
        # Finally reset metrics for next iteration
        for metric in metrics:
            metric.reset()
        
        accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)

        print(f'Epoch - {epoch} Valid-Loss : {np.mean(valid_losses[-1])} Valid-Accuracy : {accuracy}')
        
        # Finally evaluate on test data
        batch_losses=[]
        trace_y = []
        trace_yhat = []
        for i, data in enumerate(test_loader):
            x, y = data
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            trace_y.append(y.cpu().detach().numpy())
            trace_yhat.append(y_hat.cpu().detach().numpy())      
            batch_losses.append(loss.item())
        trace_y = np.concatenate(trace_y)
        trace_yhat = np.concatenate(trace_yhat)

        accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)

        print(f'Epoch - {epoch} Test-Loss : {np.mean(batch_losses)} Test-Accuracy : {accuracy}')
        
        # Before moving forward, let's flush the summary writer to ensure all data has been written
        summary_writer.flush()
        
        # Save Model Checkpoint
        with open('checkpoints/{}_{}.pth'.format(ROOT, epoch), 'wb') as f:
            torch.save({
                'model_state_dict': resnet_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f)

In [13]:
train(resnet_model, loss_fn, train_loader, valid_loader, epochs, optimizer, resnet_train_losses, resnet_valid_losses)

  0%|          | 0/50 [00:00<?, ?it/s]

Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Epoch - 1 Train-Loss : 0.736886128783226 Train-Accuracy: 0.5694444444444444
Metric Results:
Validation Accuracy is 0.5666666666666667
Validation Precision is tensor([0.5500, 0.6000], dtype=torch.float64)
Validation Recall is tensor([0.7333, 0.4000], dtype=torch.float64)
Validation MetricsLambda is 0.5542857142857143
Epoch - 1 Valid-Loss : 0.7939803898334503 Valid-Accuracy : 0.5666666666666667


  2%|▏         | 1/50 [01:32<1:15:52, 92.90s/it]

Epoch - 1 Test-Loss : 1.0072740316390991 Test-Accuracy : 0.5222222222222223
Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Epoch - 2 Train-Loss : 0.38847992817560834 Train-Accuracy: 0.8444444444444444
Metric Results:
Validation Accuracy is 0.6555555555555556
Validation Precision is tensor([0.6250, 0.7059], dtype=torch.float64)
Validation Recall is tensor([0.7778, 0.5333], dtype=torch.float64)
Validation MetricsLambda is 0.650332121819777
Epoch - 2 Valid-Loss : 0.754307895898819 Valid-Accuracy : 0.6555555555555556


  4%|▍         | 2/50 [03:06<1:14:22, 92.97s/it]

...

Epoch - 48 Test-Loss : 1.5053848028182983 Test-Accuracy : 0.6444444444444445
Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Epoch - 49 Train-Loss : 0.11847451383558412 Train-Accuracy: 0.9861111111111112
Metric Results:
Validation Accuracy is 0.5777777777777777
Validation Precision is tensor([0.5778, 0.5778], dtype=torch.float64)
Validation Recall is tensor([0.5778, 0.5778], dtype=torch.float64)
Validation MetricsLambda is 0.5777777777777777
Epoch - 49 Valid-Loss : 1.7363225817680359 Valid-Accuracy : 0.5777777777777777


 98%|█████████▊| 49/50 [1:20:51<01:47, 107.91s/it]

Epoch - 49 Test-Loss : 2.0208215713500977 Test-Accuracy : 0.5888888888888889
Index: 0
Index: 1
Index: 2
Index: 3
Index: 4
Index: 5
Index: 6
Index: 7
Index: 8
Index: 9
Index: 10
Index: 11
Epoch - 50 Train-Loss : 0.1272269010078162 Train-Accuracy: 0.9569444444444445
Metric Results:
Validation Accuracy is 0.6777777777777778
Validation Precision is tensor([0.6818, 0.6739], dtype=torch.float64)
Validation Recall is tensor([0.6667, 0.6889], dtype=torch.float64)
Validation MetricsLambda is 0.6777379923447339
Epoch - 50 Valid-Loss : 0.9523112177848816 Valid-Accuracy : 0.6777777777777778


100%|██████████| 50/50 [1:22:38<00:00, 99.18s/it] 

Epoch - 50 Test-Loss : 1.242373526096344 Test-Accuracy : 0.6333333333333333


In [14]:
# Finalize the Summary after Training
summary_writer.close()

# Review Statistics on Test Set
Including: AUC, Recall, F1, and Precision

In [ ]:
# Compute AUC

# Compute Recall

# Compute F1

# Compute Precision

# Save & Load Model

## Inference Strategy
Use this one if you're only trying to save a model for future predictions / inferences (no need to train it further)

In [ ]:
# Inference strategy
with open('coughmodel_resnet_wavegan_50.pth','wb') as f:
    torch.save(resnet_model, f)

## Checkpoint Strategy (Recommended)
Use this one if you'd like to save a model for future training

In [ ]:
# Checkpoint strategy
# https://pytorch.org/tutorials/beginner/saving_loading_models.html#saving-loading-a-general-checkpoint-for-inference-and-or-resuming-training
with open('checkpoints/{}_{}.pth'.format(ROOT, epoch), 'wb') as f:
    torch.save({
        'model_state_dict': resnet_model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f)
checkpoint = torch.load('checkpoints/{}_{}.pth')

In [ ]:
import matplotlib.pyplot as plt

tl = np.asarray(resnet_train_losses).ravel()
vl = np.asarray(resnet_valid_losses).ravel()
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.plot(tl)
plt.legend(['Train Loss'])
plt.subplot(1,2,2)
plt.plot(vl,'orange')
plt.legend(['Valid Loss'])